In [1]:
%matplotlib inline

In [2]:
# !pip install pydub torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import sys

import matplotlib.pyplot as plt
import IPython.display as ipd

from tqdm import tqdm

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [12]:
!pip install datasets[audio]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 8.1 MB/s 
     |████████████████████████████████| 132 kB 52.6 MB/s 
     |████████████████████████████████| 182 kB 62.0 MB/s 
     |████████████████████████████████| 212 kB 75.2 MB/s 
     |████████████████████████████████| 127 kB 70.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# !wget -P '/content/drive/MyDrive/18781 - Speech Recognition Project/' 'https://s3.amazonaws.com/datasets.huggingface.co/SpeechCommands/v0.02/v0.02_validation.tar.gz'
# !wget -P '/content/drive/MyDrive/18781 - Speech Recognition Project/' 'https://s3.amazonaws.com/datasets.huggingface.co/SpeechCommands/v0.02/v0.02_train.tar.gz'
# !wget -P '/content/drive/MyDrive/18781 - Speech Recognition Project/' 'https://s3.amazonaws.com/datasets.huggingface.co/SpeechCommands/v0.02/v0.02_test.tar.gz'


##Dataset

In [8]:
!mkdir train
!mkdir validation
!mkdir test

!tar -xf '/content/drive/MyDrive/18781 - Speech Recognition Project/v0.02_train.tar.gz' -C './train'
!tar -xf '/content/drive/MyDrive/18781 - Speech Recognition Project/v0.02_validation.tar.gz' -C './validation'
!tar -xf '/content/drive/MyDrive/18781 - Speech Recognition Project/v0.02_test.tar.gz' -C './test'

In [9]:
!apt -qq install -y sox

The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 7 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional disk space will be used.
Selecting previously unselected package libopencore-amrnb0:amd64.
(Reading database ... 124015 files and directories currently installed.)
Preparing to unpack .../0-libopencore-amrnb0_0.1.3-2.1_amd64.deb ...
Unpacking libopencore-amrnb0:amd64 (0.1.3-2.1) ...
Selecting previously unselected package libopencore-amrwb0:amd64.
Preparing to unpack 

###Train Dataset

In [10]:
!cd ./train/_silence_/ ; sox pink_noise.wav pink_noise.wav trim 0 1 : newfile : restart ; rm -rf pink_noise.wav
!cd ./train/_silence_/ ; sox doing_the_dishes.wav doing_the_dishes.wav trim 0 1 : newfile : restart ; rm -rf doing_the_dishes.wav
!cd ./train/_silence_/ ; sox exercise_bike.wav exercise_bike.wav trim 0 1 : newfile : restart ; rm -rf exercise_bike.wav
!cd ./train/_silence_/ ; sox dude_miaowing.wav dude_miaowing.wav trim 0 1 : newfile : restart ; rm -rf dude_miaowing.wav
!cd ./train/_silence_/ ; sox white_noise.wav white_noise.wav trim 0 1 : newfile : restart ; rm -rf white_noise.wav

!cd ./train ; mkdir _not_silence_
!cd ./train ; cp backward/* _not_silence_/ ; rm -rf backward
!cd ./train ; cp bed/* _not_silence_/ ; rm -rf bed
!cd ./train ; cp bird/* _not_silence_/ ; rm -rf bird
!cd ./train ; cp cat/* _not_silence_/ ; rm -rf cat
!cd ./train ; cp dog/* _not_silence_/ ; rm -rf dog
!cd ./train ; cp down/* _not_silence_/ ; rm -rf down
!cd ./train ; cp eight/* _not_silence_/ ; rm -rf eight
!cd ./train ; cp five/* _not_silence_/ ; rm -rf five
!cd ./train ; cp follow/* _not_silence_/ ; rm -rf follow
!cd ./train ; cp forward/* _not_silence_/ ; rm -rf forward
!cd ./train ; cp four/* _not_silence_/ ; rm -rf four
!cd ./train ; cp go/* _not_silence_/ ; rm -rf go
!cd ./train ; cp happy/* _not_silence_/ ; rm -rf happy
!cd ./train ; cp house/* _not_silence_/ ; rm -rf house
!cd ./train ; cp learn/* _not_silence_/ ; rm -rf learn
!cd ./train ; cp left/* _not_silence_/ ; rm -rf left
!cd ./train ; cp marvin/* _not_silence_/ ; rm -rf marvin
!cd ./train ; cp nine/* _not_silence_/ ; rm -rf nine
!cd ./train ; cp no/* _not_silence_/ ; rm -rf no
!cd ./train ; cp off/* _not_silence_/ ; rm -rf off
!cd ./train ; cp on/* _not_silence_/ ; rm -rf on
!cd ./train ; cp one/* _not_silence_/ ; rm -rf one
!cd ./train ; cp right/* _not_silence_/ ; rm -rf right
!cd ./train ; cp seven/* _not_silence_/ ; rm -rf seven
!cd ./train ; cp sheila/* _not_silence_/ ; rm -rf sheila
!cd ./train ; cp six/* _not_silence_/ ; rm -rf six
!cd ./train ; cp stop/* _not_silence_/ ; rm -rf stop
!cd ./train ; cp three/* _not_silence_/ ; rm -rf three
!cd ./train ; cp tree/* _not_silence_/ ; rm -rf tree
!cd ./train ; cp two/* _not_silence_/ ; rm -rf two
!cd ./train ; cp up/* _not_silence_/ ; rm -rf up
!cd ./train ; cp visual/* _not_silence_/ ; rm -rf visual
!cd ./train ; cp wow/* _not_silence_/ ; rm -rf wow
!cd ./train ; cp yes/* _not_silence_/ ; rm -rf yes
!cd ./train ; cp zero/* _not_silence_/ ; rm -rf zero

sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).
sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).
sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).


In [13]:
from datasets import load_dataset, Audio

train = load_dataset(path='/content/train')
train = train.cast_column("audio", Audio(sampling_rate=16000))

Resolving data files:   0%|          | 0/7473 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/train-7564a1ef9e039e5b/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

###Test Dataset

In [29]:
!cd ./validation/_silence_/ ; sox running_tap.wav running_tap.wav trim 0 1 : newfile : restart ; rm -rf running_tap.wav

!cd ./validation ; mkdir _not_silence_
!cd ./validation ; cp backward/* _not_silence_/ ; rm -rf backward
!cd ./validation ; cp bed/* _not_silence_/ ; rm -rf bed
!cd ./validation ; cp bird/* _not_silence_/ ; rm -rf bird
!cd ./validation ; cp cat/* _not_silence_/ ; rm -rf cat
!cd ./validation ; cp dog/* _not_silence_/ ; rm -rf dog
!cd ./validation ; cp down/* _not_silence_/ ; rm -rf down
!cd ./validation ; cp eight/* _not_silence_/ ; rm -rf eight
!cd ./validation ; cp five/* _not_silence_/ ; rm -rf five
!cd ./validation ; cp follow/* _not_silence_/ ; rm -rf follow
!cd ./validation ; cp forward/* _not_silence_/ ; rm -rf forward
!cd ./validation ; cp four/* _not_silence_/ ; rm -rf four
!cd ./validation ; cp go/* _not_silence_/ ; rm -rf go
!cd ./validation ; cp happy/* _not_silence_/ ; rm -rf happy
!cd ./validation ; cp house/* _not_silence_/ ; rm -rf house
!cd ./validation ; cp learn/* _not_silence_/ ; rm -rf learn
!cd ./validation ; cp left/* _not_silence_/ ; rm -rf left
!cd ./validation ; cp marvin/* _not_silence_/ ; rm -rf marvin
!cd ./validation ; cp nine/* _not_silence_/ ; rm -rf nine
!cd ./validation ; cp no/* _not_silence_/ ; rm -rf no
!cd ./validation ; cp off/* _not_silence_/ ; rm -rf off
!cd ./validation ; cp on/* _not_silence_/ ; rm -rf on
!cd ./validation ; cp one/* _not_silence_/ ; rm -rf one
!cd ./validation ; cp right/* _not_silence_/ ; rm -rf right
!cd ./validation ; cp seven/* _not_silence_/ ; rm -rf seven
!cd ./validation ; cp sheila/* _not_silence_/ ; rm -rf sheila
!cd ./validation ; cp six/* _not_silence_/ ; rm -rf six
!cd ./validation ; cp stop/* _not_silence_/ ; rm -rf stop
!cd ./validation ; cp three/* _not_silence_/ ; rm -rf three
!cd ./validation ; cp tree/* _not_silence_/ ; rm -rf tree
!cd ./validation ; cp two/* _not_silence_/ ; rm -rf two
!cd ./validation ; cp up/* _not_silence_/ ; rm -rf up
!cd ./validation ; cp visual/* _not_silence_/ ; rm -rf visual
!cd ./validation ; cp wow/* _not_silence_/ ; rm -rf wow
!cd ./validation ; cp yes/* _not_silence_/ ; rm -rf yes
!cd ./validation ; cp zero/* _not_silence_/ ; rm -rf zero

sox WARN trim: Last 1 position(s) not reached (audio shorter than expected).


In [28]:
!cd ./test ; mkdir _not_silence_
!cd ./test ; cp backward/* _not_silence_/ ; rm -rf backward
!cd ./test ; cp bed/* _not_silence_/ ; rm -rf bed
!cd ./test ; cp bird/* _not_silence_/ ; rm -rf bird
!cd ./test ; cp cat/* _not_silence_/ ; rm -rf cat
!cd ./test ; cp dog/* _not_silence_/ ; rm -rf dog
!cd ./test ; cp down/* _not_silence_/ ; rm -rf down
!cd ./test ; cp eight/* _not_silence_/ ; rm -rf eight
!cd ./test ; cp five/* _not_silence_/ ; rm -rf five
!cd ./test ; cp follow/* _not_silence_/ ; rm -rf follow
!cd ./test ; cp forward/* _not_silence_/ ; rm -rf forward
!cd ./test ; cp four/* _not_silence_/ ; rm -rf four
!cd ./test ; cp go/* _not_silence_/ ; rm -rf go
!cd ./test ; cp happy/* _not_silence_/ ; rm -rf happy
!cd ./test ; cp house/* _not_silence_/ ; rm -rf house
!cd ./test ; cp learn/* _not_silence_/ ; rm -rf learn
!cd ./test ; cp left/* _not_silence_/ ; rm -rf left
!cd ./test ; cp marvin/* _not_silence_/ ; rm -rf marvin
!cd ./test ; cp nine/* _not_silence_/ ; rm -rf nine
!cd ./test ; cp no/* _not_silence_/ ; rm -rf no
!cd ./test ; cp off/* _not_silence_/ ; rm -rf off
!cd ./test ; cp on/* _not_silence_/ ; rm -rf on
!cd ./test ; cp one/* _not_silence_/ ; rm -rf one
!cd ./test ; cp right/* _not_silence_/ ; rm -rf right
!cd ./test ; cp seven/* _not_silence_/ ; rm -rf seven
!cd ./test ; cp sheila/* _not_silence_/ ; rm -rf sheila
!cd ./test ; cp six/* _not_silence_/ ; rm -rf six
!cd ./test ; cp stop/* _not_silence_/ ; rm -rf stop
!cd ./test ; cp three/* _not_silence_/ ; rm -rf three
!cd ./test ; cp tree/* _not_silence_/ ; rm -rf tree
!cd ./test ; cp two/* _not_silence_/ ; rm -rf two
!cd ./test ; cp up/* _not_silence_/ ; rm -rf up
!cd ./test ; cp visual/* _not_silence_/ ; rm -rf visual
!cd ./test ; cp wow/* _not_silence_/ ; rm -rf wow
!cd ./test ; cp yes/* _not_silence_/ ; rm -rf yes
!cd ./test ; cp zero/* _not_silence_/ ; rm -rf zero

In [30]:
!mv validation/_silence_/* test/_silence_/
!mv validation/_not_silence_/* test/_not_silence_/ ; rm -rf validation

In [31]:
test = load_dataset(path='/content/test')
test = test.cast_column("audio", Audio(sampling_rate=16000))

Resolving data files:   0%|          | 0/2064 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset audiofolder downloaded and prepared to /root/.cache/huggingface/datasets/audiofolder/test-ebf37508ff66cd0e/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
train_set = train['train']
test_set = test['train']

In [16]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    batch = torch.unsqueeze(batch, dim=1)
    return batch


def collate_fn(batch):

    # A data tuple has the form:
    # waveform, sample_rate, label, speaker_id, utterance_number

    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for i in batch:
      tensors += [torch.from_numpy(i['audio']['array'])]
      targets += [torch.tensor(i['label'])]

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)

    return tensors, targets

In [17]:
batch_size = 32

In [18]:
if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    collate_fn=collate_fn,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

In [4]:
class M5(nn.Module):
    def __init__(self, n_input=1, n_output=2, stride=16, n_channel=32):
        super().__init__()
        self.conv1 = nn.Conv1d(n_input, n_channel, kernel_size=80, stride=stride)
        self.bn1 = nn.BatchNorm1d(n_channel)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(n_channel, n_channel, kernel_size=3)
        self.bn2 = nn.BatchNorm1d(n_channel)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(n_channel, 2 * n_channel, kernel_size=3)
        self.bn3 = nn.BatchNorm1d(2 * n_channel)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(2 * n_channel, 2 * n_channel, kernel_size=3)
        self.bn4 = nn.BatchNorm1d(2 * n_channel)
        self.pool4 = nn.MaxPool1d(4)
        self.fc1 = nn.Linear(2 * n_channel, n_output)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = F.avg_pool1d(x, x.shape[-1])
        x = x.permute(0, 2, 1)
        x = self.fc1(x)
        return x


model = M5(n_input=1, n_output=2)
model.to(device)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


n = count_parameters(model)
print("Number of parameters: %s" % n)

M5(
  (conv1): Conv1d(1, 32, kernel_size=(80,), stride=(16,))
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(64, 64, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=64, out_features=2, bias=True)
)
Number

In [20]:
# zero_count = 0
# one_count = 1

# for datapoint in train_set:
#   if datapoint['label']==0:
#     zero_count +=1
#   elif datapoint['label']==1:
#     one_count +=1

In [21]:
# print("0 :", zero_count, "1 :", one_count)

# total = (zero_count + one_count)
# weight_vector = torch.tensor([1, zero_count/one_count])

weight_vector = torch.tensor([1,21]).cuda()
weight_vector

tensor([ 1, 21], device='cuda:0')

In [22]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)  # reduce the learning after 20 epochs by a factor of 10
loss_fn = nn.BCEWithLogitsLoss(pos_weight=weight_vector)

In [23]:
def train(model, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        # data = transform(data)
        output = model(data)


        # print(output.shape, target.shape)
        # negative log-likelihood for a tensor of size (batch x 1 x n_output)
        loss = nn.CrossEntropyLoss(reduction = "none")(output.squeeze(), target.long())
        loss[target == 1] *= 21
        loss = loss.mean()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print training stats
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

        # update progress bar
        pbar.update(pbar_update)
        # record loss
        losses.append(loss.item())

In [19]:
def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


In [25]:
def test(model, epoch, best_acc):
    model.eval()
    correct = 0
    for data, target in test_loader:

        data = data.to(device)
        target = target.to(device)

        # apply transform and model on whole batch directly on device
        output = model(data)

        pred = get_likely_index(output)
        correct += number_of_correct(pred, target)

        # update progress bar
        pbar.update(pbar_update)

    if correct / len(test_loader.dataset) > best_acc:
      print("Saving model..")
      PATH = "/content/drive/MyDrive/18781 - Speech Recognition Project/model.pt"

      torch.save({
                  'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  }, PATH)

    print(f"\nTest Epoch: {epoch}\tAccuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")
    return best_acc

In [26]:
log_interval = 20
n_epoch = 15

pbar_update = 1 / (len(train_loader) + len(test_loader))
best_acc = 0
losses = []

# The transform needs to live on the same device as the model and the data.
with tqdm(total=n_epoch) as pbar:
    for epoch in range(1, n_epoch + 1):
        train(model, epoch, log_interval)
        best_acc = test(model, epoch, best_acc)
        scheduler.step()

# Let's plot the training loss versus the number of iteration.
# plt.plot(losses);
# plt.title("training loss");



  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0.0033444816053511705/15 [00:09<12:06:48, 2907.91s/it]

  0%|          | 0.016722408026755852/15 [00:09<1:49:55, 440.19s/it]   

Train Epoch: 1 [0/7473 (0%)]	Loss: 1.761357




  0%|          | 0.03010033444816054/15 [00:09<50:43, 203.28s/it]   

  0%|          | 0.043478260869565216/15 [00:10<29:16, 117.47s/it]

  0%|          | 0.05685618729096989/15 [00:10<18:49, 75.58s/it]  

  0%|          | 0.07023411371237456/15 [00:10<12:50, 51.62s/it]

  1%|          | 0.08361204013377924/15 [00:10<09:15, 37.27s/it]

Train Epoch: 1 [640/7473 (9%)]	Loss: 0.846571




  1%|          | 0.09698996655518391/15 [00:10<06:55, 27.90s/it]

  1%|          | 0.11036789297658858/15 [00:10<05:21, 21.61s/it]

  1%|          | 0.12374581939799326/15 [00:10<04:19, 17.46s/it]

  1%|          | 0.13712374581939793/15 [00:10<03:35, 14.51s/it]

  1%|          | 0.1505016722408026/15 [00:10<03:05, 12.51s/it] 

...

Train Epoch: 15 [6400/7473 (85%)]	Loss: 1.164025




 98%|█████████▊| 14.70568561872993/15 [02:06<00:02,  8.16s/it] 

 98%|█████████▊| 14.719063545151336/15 [02:06<00:02,  8.30s/it]

 98%|█████████▊| 14.732441471572741/15 [02:06<00:02,  8.29s/it]

 98%|█████████▊| 14.745819397994147/15 [02:06<00:02,  8.29s/it]

 98%|█████████▊| 14.759197324415553/15 [02:06<00:01,  8.15s/it]

Train Epoch: 15 [7040/7473 (94%)]	Loss: 0.396221




 98%|█████████▊| 14.772575250836958/15 [02:06<00:01,  8.22s/it]

 99%|█████████▊| 14.785953177258364/15 [02:07<00:01,  8.07s/it]

 99%|█████████▊| 14.79933110367977/15 [02:07<00:01,  8.01s/it] 

 99%|█████████▉| 14.816053511706526/15 [02:07<00:01,  7.59s/it]

 99%|█████████▉| 14.832775919733283/15 [02:07<00:01,  7.58s/it]

 99%|█████████▉| 14.84949832776004/15 [02:07<00:01,  7.27s/it] 

 99%|█████████▉| 14.866220735786797/15 [02:07<00:00,  7.03s/it]

 99%|█████████▉| 14.882943143813554/15 [02:07<00:00,  7.19s/it]

 99%|█████████▉| 14.899665551840311/15 [02:07<00:00,  6.91s/it]

 99%|█████████▉| 14.916387959867068/15 [02:07<00:00,  6.88s/it]

100%|█████████▉| 14.933110367893825/15 [02:08<00:00,  6.90s/it]

100%|█████████▉| 14.949832775920582/15 [02:08<00:00,  6.85s/it]

100%|█████████▉| 14.966555183947339/15 [02:08<00:00,  6.88s/it]

100%|█████████▉| 14.983277591974096/15 [02:08<00:00,  6.81s/it]/usr/local/lib/python3.8/dist-packages/tqdm/std.py:533: TqdmWarning: clamping frac to rang

Saving model..

Test Epoch: 15	Accuracy: 1983/2064 (96%)



In [33]:
def predict(tensor):
  # Use the model to predict the label of the waveform
  tensor = tensor.to(device)
  tensor = model(tensor.unsqueeze(0))
  tensor = get_likely_index(tensor)
  return tensor


datapoint = train_set[40]
print(f"Expected: ,", datapoint["label"], ". Predicted: ",predict(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)),".")

datapoint = train_set[-1]
print(f"Expected: ,", datapoint["label"], ". Predicted: ",predict(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)),".")


Expected: , 0 . Predicted:  tensor([[0]]) .
Expected: , 1 . Predicted:  tensor([[1]]) .


In [35]:
model = M5(n_input=1, n_output=2)
model.load_state_dict(torch.load('/content/drive/MyDrive/18781 - Speech Recognition Project/Saved Model/model.pt', map_location=torch.device('cpu'))['model_state_dict'])
model.to(device)
model.eval()

model_scripted = torch.jit.script(model) 
model_scripted = torch.jit.load('/content/drive/MyDrive/18781 - Speech Recognition Project/Saved Model/model_scripted.pt')

def predict_scripted(tensor):
  # Use the model to predict the label of the waveform
  tensor = tensor.to(device)
  tensor = model_scripted(tensor.unsqueeze(0))
  tensor = get_likely_index(tensor)
  return tensor


datapoint = train_set[40]
print(f"Expected: ,", datapoint["label"], ". Predicted: ", predict_scripted(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)),".")

datapoint = train_set[-1]
print(f"Expected: ,", datapoint["label"], ". Predicted: ", predict(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)),". \nScripted Predicted: ",predict_scripted(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)),".")


Expected: , 0 . Predicted:  tensor([[0]]) .
Expected: , 1 . 
Predicted:  tensor([[1]]) . 
Scripted Predicted:  tensor([[1]]) .


In [7]:
model_scripted.save('/content/drive/MyDrive/18781 - Speech Recognition Project/Saved Model/model_scripted.pt')

In [ ]:
for i in range(len(train_set)):
  try:
    datapoint = train_set[i]
    print(f"Expected: ,", datapoint["label"], ". \nPredicted: ", predict(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)).item(),". \nScripted Predicted: ",predict_scripted(torch.tensor(datapoint["audio"]["array"]).unsqueeze(dim=0)).item(),".")
    print('-----------------------------------------------------')
  except:
    continue